## Plotting maps

This notebook is about plotting maps (and points on map) using different packages. I used this code a lot of times to build maps and I think can be helpful as an initial guide to the ones that are learning the basics. The examples here will use different packages:

- Folium: https://github.com/python-visualization/folium
- Plotting Shapefiles with Matplotlib and Geopandas: http://geopandas.org/mapping.html

All the code compiled in this notebook came from documentation examples and might have been changed a little bit to fit at my purposes in the plot.


In [4]:
from __future__ import print_function
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from geopy.geocoders import Nominatim

%matplotlib inline

### Example 1: Plotting points in a map with Folium

Code inspired in a workshop taught by https://github.com/polong-lin (IBM data science instructor)

Given addresses or latitude/longitude, Folium can plot points on a map in a beautiful manner, allowing the user to change the basemap. To this example, we are also going to transform an address into latitude and longitude point using Nominatim package.


In [20]:
# given addresses, let's get latitude and longitude of those
address_U1 = "960 Madison Avenue New York, NY"
address_U2 = "1060 Madison Avenue New York, NY"

# Geolocator translate addrss into lat, lon
geolocator = Nominatim()
location_U1 = geolocator.geocode(address_U1)
location_U2 = geolocator.geocode(address_U2)
latitude_U1, longitude_U1 = (location_U1.latitude, location_U1.longitude)
latitude_U2, longitude_U2 = (location_U2.latitude, location_U2.longitude)
print (latitude_U1, longitude_U1, latitude_U2, longitude_U2)

40.7739694 -73.9636632 40.777307375 -73.96153


In [37]:
# Create a map centered in a location chosen by the user (in this case 40.765937,-73.977304)
map_nyc = folium.Map(location=[40.7739694, -73.9636632], zoom_start=13)

# For every point to be plotted, add its location and if desired, a label can be shown if you click on the point
folium.CircleMarker([latitude_U1, longitude_U1], color = 'blue',
                    radius = 7, popup="John's House").add_to(map_nyc)
folium.CircleMarker([latitude_U2, longitude_U2], color = 'green',
                    radius = 7, popup="Jack's House").add_to(map_nyc)

map_nyc

- Adding marker to points;
- Creating for loop to create points instead of coding a line for each point;

Let's say that we want to plot a restaurants near a certain point, and show them with markers and when clicked show the name of the restaurant

In [29]:
# This dataset I got from Foursquare API. 
restaurants = pd.read_csv('Data/restaurants.csv')

In [30]:
restaurants.head()

,Unnamed: 0,name,url,categories,rating,id,isOpen,tier,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state
0,1,Kafana,http://www.kafananyc.com,b'Eastern European Restaurant',8.9,49e8896cf964a5204e651fe3,False,3,116 Avenue C,US,New York,United States,at E 8th St,312,"['116 Avenue C (at E 8th St)', 'New York, NY 1...","[{'lat': 40.72445009594483, 'lng': -73.9786382...",40.724450,-73.978638,10009.0,NY
1,11,Gruppo,http://www.gruppothincrust.com/,b'Pizza Place',8.8,3fd66200f964a5203ce51ee3,False,2,98 Avenue B,US,New York,United States,btwn 6th & 7th Sts,499,"['98 Avenue B (btwn 6th & 7th Sts)', 'New York...","[{'lat': 40.724949645619056, 'lng': -73.981332...",40.724950,-73.981333,10009.0,NY
2,12,Ciao For Now,http://ciaofornow.net/?page=home,b'Caf',8.6,3fd66200f964a5203fe51ee3,False,2,523 E 12th St,US,New York,United States,btwn Avenue B & Avenue A,491,"['523 E 12th St (btwn Avenue B & Avenue A)', '...","[{'lat': 40.728625692654504, 'lng': -73.980319...",40.728626,-73.980320,10009.0,NY
3,16,Eleven B,http://www.elevenbnyc.com,b'Italian Restaurant',6.5,4ae1b55df964a520df8621e3,False,2,174 Avenue B,US,New York,United States,11th Street,366,"['174 Avenue B (11th Street)', 'New York, NY 1...","[{'lat': 40.727518, 'lng': -73.979478, 'label'...",40.727518,-73.979478,10009.0,NY
4,29,Housewarmings!,NaN,b'Gastropub',NaN,515392c4e4b0f03a43c69fce,NaN,3,NaN,US,New York,United States,NaN,493,"['New York, NY', 'United States']","[{'lat': 40.72523709670373, 'lng': -73.9813471...",40.725237,-73.981347,NaN,NY


In [32]:
# Code inspired on material from IBM Workshop by Alex Aklson - https://www.linkedin.com/in/aklson/
venues_map = folium.Map(location=[40.724450, -73.978638], zoom_start=15) # generate map

# Creating restaurants on map
places = folium.map.FeatureGroup()
for lat, lng, in zip(restaurants.lat, restaurants.lng):
    places.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            color="blue",
            fill_color="blue",
            fill_opacity=0.6)
        )
venues_map.add_child(places)


# add labels to each point on the map
latitudes_list = list(restaurants.lat)
longitudes_list = list(restaurants.lng)
labels = list(restaurants.name)

for lat, lng, label in zip(latitudes_list, longitudes_list, labels):
    folium.Marker([lat, lng], popup=label).add_to(venues_map)

# display map
venues_map